In [2]:
#import every libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import PIL
import cv2

from tensorflow import keras
from tensorflow.keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.optimizers import *
from tensorflow.keras import *
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D,BatchNormalization,Input, Lambda
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

In [3]:
#fetch the datasets from google drive
train_path = 'C:\\Users\\DELL\\Desktop\\SDP\\Dataset\\train'
valid_path = 'C:\\Users\\DELL\\Desktop\\SDP\\Dataset\\test'

In [4]:
# add preprocessing layer to the front of VGG
#transfer learning - VGG 16 / RGB images / remove the final layer
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [5]:
# freeze the all weightings of middle layers
for layer in vgg.layers:
  layer.trainable = False

In [6]:
# find the number of classes
from glob import glob
folders = glob('C:\\Users\\DELL\\Desktop\\SDP\\Dataset\\train\\*')
print(len(folders))

250


In [7]:
x = Flatten()(vgg.output)

In [8]:
prediction = Dense(len(folders), activation='softmax')(x)

In [9]:
model = Model(inputs=vgg.input, outputs=prediction)

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [11]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [12]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 35255 images belonging to 250 classes.
Found 1250 images belonging to 250 classes.


In [13]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

C:\E-Space\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
1102/1102 [==============================] - 6190s 6s/step - loss: 5.2832 - accuracy: 0.2686 - val_loss: 2.1731 - val_accuracy: 0.6392
Epoch 2/20
1102/1102 [==============================] - 4008s 4s/step - loss: 2.1647 - accuracy: 0.6400 - val_loss: 2.1262 - val_accuracy: 0.7000
Epoch 3/20
1102/1102 [==============================] - 3978s 4s/step - loss: 1.7577 - accuracy: 0.7156 - val_loss: 1.9215 - val_accuracy: 0.7288
Epoch 4/20
1102/1102 [==============================] - 3952s 4s/step - loss: 1.4346 - accuracy: 0.7711 - val_loss: 1.8455 - val_accuracy: 0.7640
Epoch 5/20
1102/1102 [==============================] - 3948s 4s/step - loss: 1.2456 - accuracy: 0.7960 - val_loss: 1.8626 - val_accuracy: 0.7592
Epoch 6/20
1102/1102 [==============================] - 3653s 3s/step - loss: 1.1411 - accuracy: 0.8207 - val_loss: 1.8627 - val_accuracy: 0.7856
Epoch 7/20
1102/1102 [==============================] - 3650s 3s/step - loss: 0.9626 - accuracy: 0.8440 - val_loss: 2.0072 -

In [3]:
from tensorflow.keras.models import load_model
model.save('birdo_model.h5')

NameError: name 'model' is not defined

In [4]:
from keras.models import load_model
birdo_model = load_model('birdo_model.h5')

In [7]:
import matplotlib.pyplot as plt
%matplotlib inline
#list all data in training
print(birdo_model.history.keys())
# summarize training for accuracy
plt.plot(birdo_model.history['accuracy'])
plt.plot(birdo_model.history['val_accuracy'])
plt.title('Accuracy graph')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['train','test'], loc='upper left')
plt.show()

# summarize training for loss
plt.plot(birdo_model.history['loss'])
plt.plot(birdo_model.history['val_loss'])
plt.title('Loss graph')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train','test'], loc='upper left')
plt.show()

AttributeError: 'NoneType' object has no attribute 'keys'